# Lab Week 8 Task 2 Code
This task is concerned with creating a dog/horse classifier using the [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset.

This is a bit more advanced than task 1 as it will show an example of how custom training works in neural networks, rather than just calling the *model.fit()* method.



**TASK**: You are required to look up any function calls that are unclear to you to understand them: https://www.tensorflow.org/api_docs/python/tf/keras


**NOTE**: Some parts of the code are outlined with the keyword `ADVANCED CODE`. You do not need to try to understand what this part of the code does, simply read the comment next to it.

In [ ]:
# import modules
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

CIFAR 10 is a dataset of 60,000 32x32pixel color images in 10 classes, with 6,000 images per class. There are 50,000 training images and 10,000 test images.

Therefore, the dataset is evently distributed among the 10 classes.

We are using a dataset with small images to enable training on your computers.

In [ ]:
# load data
# depending on your internet connection, this may take a while
cifar10 = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()


In [ ]:
# we use the class names to label the images as the dataset does not contain class names.
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print(train_images.shape)
print(test_images.shape)

# notice that we have 3 channels, one for each color

The following code shows samples of the images in the original dataset with their corresponding labels

In [ ]:
# ADVANCED CODE: Plot the 16 random images from the training set and display the class name below each image.

def sample_images(images, labels, class_names):
    plt.figure(figsize=(5, 5))

    # get 16 random numbers between 0 and the number of images
    random_indices = np.random.randint(0, len(images), 16)
    

    for i in range(16):
        image_ix = random_indices[i]
        plt.subplot(4, 4, i + 1)
        plt.subplots_adjust(hspace=.3)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[image_ix], cmap=plt.cm.binary)
        plt.title(class_names[labels[image_ix][0]])
    plt.show()

sample_images(train_images, train_labels, class_names)

# Horses vs Dogs Classifier

For this use-case we simply want to build a Horse-Dog classifier. 
The following cell contains code that extracts all the horses and all the dogs, from both the training and testing dataset and creates the new datasets with only those two classes. 

In [ ]:
# ADVANCED CODE: Extract all images with horses and dogs from the training set and testing set and use them as our new training and testing set.

# filter out the images and labels for just horses and dogs
horse_images_train = train_images[train_labels[:, 0] == 7]
horse_labels_train = train_labels[train_labels[:, 0] == 7]
dog_images_train = train_images[train_labels[:, 0] == 5]
dogs_labels_train = train_labels[train_labels[:, 0] == 5]

horse_images_test = test_images[test_labels[:, 0] == 7]
horse_labels_test = test_labels[test_labels[:, 0] == 7]
dog_images_test = test_images[test_labels[:, 0] == 5]
dogs_labels_test = test_labels[test_labels[:, 0] == 5]

# combine the horse and dog images into a single array for train and test
train_images = np.concatenate((horse_images_train, dog_images_train))
test_images = np.concatenate((horse_images_test, dog_images_test))

# combine the horse and dog labels into a single array for train and test
train_labels = np.concatenate((horse_labels_train, dogs_labels_train))
test_labels = np.concatenate((horse_labels_test, dogs_labels_test))

# create a new array of labels that are either 0 or 1
# 0 for horse and 1 for dog
train_labels = np.where(train_labels == 7, 0, 1)
test_labels = np.where(test_labels == 7, 0, 1)

class_names = ['horse', 'dog']

sample_images(train_images, train_labels, class_names)


Images need to be rescaled to be between 0 and 1 before being fed into the neural network. This is necessary because the pixel values are currently between 0 and 255 and neural networks work better with smaller input values, therefore we divide the values (which were between 0-255) by 255. 

**NOTE**: Do not rescale the labels!

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# we set a random seed so that the results are reproducible
tf.random.set_seed(42)

# The Model

**TASK**: Use Task 1 as inspiration or your knowledge from last week to build a small, convolutional neural network following this specification:


- Convolutional Layer, 3x3 Kernel, 32 Filters, 32px * 32px * 3 Input Size
- ReLu Activation
- Max Pooling Operation
- Convolutional Layer, 3x3 Kernel, 32 Filters
- ReLu Activation
- Max Pooling Operation
- Convolutional Layer, 3x3 Kernel, 32 Filters
- ReLu Activation
- Global Average Pooling Operation [Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling2D). (This layer combines each feature channel into a single value by averaging it).
- Fully Connected Layer, 64 Units
- ReLu Activation
- Fully Connected Layer, 2 Units
- Softmax Activation

Then print the summary of this model. 


In [ ]:
from tensorflow.keras import datasets, layers, models, losses


model = # TODO: Your code here

Before training with the model, you need to compile it and assign it a loss and optimizer.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

The following code fits the model to the training images for 10 epochs using a batch size of 32

In [ ]:
model.fit(train_images, train_labels, batch_size=32, epochs=20)

The model.evaluate() method checks the models performance
However, as we have learned in the lecture about metrics, accuracy is not always the best metric to use

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('Accuracy on test set:', test_acc)

Generally we would like to also see the confusion matrix and further metrics.
Code for this is implemented in the next cell. 

In [ ]:
# ADVANCED CODE: Code to plot the confusion matrix and print the f1, precision, and recall scores

from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import seaborn as sns

def plot_print_metrics(model, test_images, test_labels, class_names): 
    # create predictions
    predictions = model.predict(test_images)



    # get the predicted class as the index of the highest probability
    predicted_classes = np.argmax(predictions, axis=1)

    # calculate the precision, recall, and f1 score
    precision = precision_score(test_labels, predicted_classes)
    recall = recall_score(test_labels, predicted_classes)
    f1 = f1_score(test_labels, predicted_classes)

    # print the precision, recall, and f1 score
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1 Score:', f1)

    # create the confusion matrix
    cm = confusion_matrix(test_labels, predicted_classes)

    # plot the confusion matrix
    plt.figure(figsize=(3, 3))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()  




plot_print_metrics(model, test_images, test_labels, class_names)

# Custom Training Process and Comparison with Augmentations

Augmentations are a very strong tool that can be used in deep neural networks, especially with image data.
There are several different ways on how to perform image augmentation, one was shown in the Task 1. 
The availablility of different augmentation libraries allows a high flexibilyt and adaptability to your use case. 
Popular libraries for this are:
- PIL (Python Image Library) [Link](https://pillow.readthedocs.io/en/stable/)
- Albumentations [Link](https://albumentations.ai/docs/)
- Keras Preprocessing [Link](https://keras.io/guides/preprocessing_layers/)


**TASK**: Ensure that you have installed the Albumentations library in your anaconda environment using the command: `pip install albumentations` 

In [ ]:
# use the albumentations library to perform data augmentation
import albumentations as A


# create a transform object that will perform the data augmentation
# a list of abailable transforms can be found here: https://albumentations.ai/docs/api_reference/augmentations/transforms/
# this randomly flips the image horizontally and vertically with a probability of 0.5
# it also randomly shifts, resizes and rotates the image randomly up to 10° with a probability of 0.5
# and it randomly cuts out a 2 portions of the image with a probability of 0.5
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5, rotate_limit=10),
    A.CoarseDropout(p=0.5, max_holes=2),
])


# ADVANCED CODE: A function that applies the transform with our augmentations to the image or multiple images
def apply_augmentation(images, transform):
    # function to apply the augmentation in the transform to the images

    # this library needs images to be in range 0-255
    images = (images * 255).astype(np.uint8)

    if len(images.shape) == 4:
        # if there are multiple images, apply the augmentation to each image
        augmented_images = []
        for image in images:
            augmented_images.append(transform(image=image)['image'])
        augmented_images = np.array(augmented_images)
    else:
        # if there is only one image, apply the augmentation to the image
        augmented_images = transform(image=images)['image']
        augmented_images = np.array(augmented_images)


    # convert the images back to a range of 0-1
    augmented_images = augmented_images / 255.

    return augmented_images

The following cell shows some example code that shows examples of our proposed augmentation strategy. 

In [ ]:
# ADVANCED CODE:  Function to show a random image from the training set augmented
def show_augmentation(augmentation_function, original_image):
    plt.figure(figsize=(5, 5))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.subplots_adjust(hspace=.3)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        if i == 0:
            augmented_image = original_image
            plt.title('Original')
        else:
            plt.title('Augmented')
            augmented_image = augmentation_function(original_image, transform)

        plt.imshow(augmented_image)
    plt.show()


show_augmentation(apply_augmentation, train_images[np.random.randint(0, len(train_images))])

We then want to recreate the model without copying the trained weights. The same settings as before are utilized to show a fair comparison with the non-augmented model. 

In [ ]:
# cloning the model replicates the model architecture without the weights that have been trained in the previous cells
model_aug = tf.keras.models.clone_model(model)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.losses.SparseCategoricalCrossentropy()
model_aug.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

## Custom Training Loop
The following code shows how tensorflow and keras can be used to create a custom training loop.
Remember from the deep learning lecture, during training we have:
- Iterations: A single forward and backward pass of one or more samples through a neural network where the weights are updated
- Batch: A number of paralell data samples that are used in an iteration
- Epoch: One pass through the entire dataset using the batches. The total number of epochs is set by *you* but the number of iterations in an epoch is the number of samples in the dataset (`steps_per_epoch`), divided by the number of samples in a single batch. 

There is no right or wrong, whether you use a custom training loop as below or not. The example below offers you better capabilities, such as choosing your own augmentaiton provider. In most use cases you can however stick to the ones provied by the framework you are using.

The example below is just to visualize (in code) how batches/epochs/iterations work.

In [ ]:
# custom keras training loop
epochs = 20
batch_size = 32

steps_per_epoch = len(train_images) // batch_size

# run the training process for the specified number of epochs
for e in range(epochs):

    # get random indexes to select random samples at each epoch
    # instead of picking the samples directly, we pick the indexes which allow us to pick the correct samples and their labels. 
    indexes = np.random.permutation(len(train_images))

    # keeping track of the loss and metric for this epoch
    epoch_loss = 0
    epoch_metric = 0

    # iterate over the indexes in batches
    for i in range(0, len(indexes), batch_size):

        # get the next batch of indexes
        batch_indexes = indexes[i:i+batch_size]

        # get the images and labels for the current batch
        # indexing allows us to get the correct samples and their labels
        batch_images = train_images[batch_indexes]
        batch_labels = train_labels[batch_indexes]

        # augment the images using your data augmentation pipeline and preferred strategy 
        # we are using albumentations
        batch_images = apply_augmentation(batch_images, transform)

        # train the model on this current batch
        loss, metric = model_aug.train_on_batch(batch_images, batch_labels)

        # update the epoch loss and metric by adding the loss and metric of the current batch
        epoch_loss += loss
        epoch_metric += metric

    # print the loss and metric for this epoch 
    print(f'Epoch {e+1} completed', 'Epoch Loss:', epoch_loss/steps_per_epoch, 'Epoch Metric:', epoch_metric/steps_per_epoch)


In [ ]:
test_loss, test_acc = model_aug.evaluate(test_images, test_labels, verbose=2)

print('Accuracy on test set:', test_acc)

We can now compare our *baseline* model with the *augmented* model. 

In [ ]:
print("Baseline Model")
plot_print_metrics(model, test_images, test_labels, class_names)
print("Augmented Model")
plot_print_metrics(model_aug, test_images, test_labels, class_names)

# Further Exercises
3. Read the guide on how to create an augmentation pipeline from Albumentations [here](https://albumentations.ai/docs/examples/example/). Then add another sample augmentation such as RandomBrighntessContrast. If you retrain the models, does this change the performance?
4. Try to use a larger model for training. For example, instead of only having a single convolutional layer, try using two with ReLU activations inbetween. This offers better feature extraction capabilities. Check if this improves performance.